## Deliverable 2. Create a Customer Travel Destinations Map.

In [8]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Bluff,-46.6000,168.3333,59.32,87,100,7.52,NZ,2022-01-30 01:24:35,overcast clouds
1,1,Toktogul,41.8708,72.9402,22.60,72,36,2.80,KG,2022-01-30 01:25:26,scattered clouds
2,2,Jamestown,42.0970,-79.2353,4.71,85,0,8.05,US,2022-01-30 01:24:19,clear sky
3,3,Hokitika,-42.7167,170.9667,67.71,61,79,9.42,NZ,2022-01-30 01:24:40,broken clouds
4,4,Busselton,-33.6500,115.3333,77.09,60,7,13.00,AU,2022-01-30 01:19:43,clear sky


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 30
What is the maximum temperature you would like for your trip? 90


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Bluff,-46.6000,168.3333,59.32,87,100,7.52,NZ,2022-01-30 01:24:35,overcast clouds
3,3,Hokitika,-42.7167,170.9667,67.71,61,79,9.42,NZ,2022-01-30 01:24:40,broken clouds
4,4,Busselton,-33.6500,115.3333,77.09,60,7,13.00,AU,2022-01-30 01:19:43,clear sky
5,5,Isangel,-19.5500,169.2667,78.91,94,100,6.91,VU,2022-01-30 01:24:37,drizzle
7,7,Barra Do Garcas,-15.8900,-52.2567,71.82,96,100,4.32,BR,2022-01-30 01:25:27,light rain
8,8,Lebu,-37.6167,-73.6500,58.41,88,0,9.04,CL,2022-01-30 01:22:09,clear sky
9,9,Lasa,46.6166,10.7002,47.66,81,15,9.01,IT,2022-01-30 01:15:41,few clouds
10,10,Castro,-24.7911,-50.0119,59.86,99,100,6.51,BR,2022-01-30 01:24:43,overcast clouds
12,12,Nkowakowa,-23.8878,30.2871,60.85,89,100,1.19,ZA,2022-01-30 01:25:28,overcast clouds
13,13,Saint-Philippe,-21.3585,55.7679,76.55,83,75,10.29,RE,2022-01-30 01:25:28,broken clouds


In [15]:
# 4a. Determine if there are any empty rows.
if preferred_cities_df.empty == True:
    print('DataFrame is empty')
else:
    print("DataFrame is not empty")

DataFrame is not empty


In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bluff,NZ,59.32,overcast clouds,-46.6000,168.3333,
3,Hokitika,NZ,67.71,broken clouds,-42.7167,170.9667,
4,Busselton,AU,77.09,clear sky,-33.6500,115.3333,
5,Isangel,VU,78.91,drizzle,-19.5500,169.2667,
7,Barra Do Garcas,BR,71.82,light rain,-15.8900,-52.2567,
8,Lebu,CL,58.41,clear sky,-37.6167,-73.6500,
9,Lasa,IT,47.66,few clouds,46.6166,10.7002,
10,Castro,BR,59.86,overcast clouds,-24.7911,-50.0119,
12,Nkowakowa,ZA,60.85,overcast clouds,-23.8878,30.2871,
13,Saint-Philippe,RE,76.55,broken clouds,-21.3585,55.7679,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [23]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bluff,NZ,59.32,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
3,Hokitika,NZ,67.71,broken clouds,-42.7167,170.9667,Beachfront Hotel Hokitika
4,Busselton,AU,77.09,clear sky,-33.6500,115.3333,Observatory Guest House
5,Isangel,VU,78.91,drizzle,-19.5500,169.2667,Tanna Lodge
7,Barra Do Garcas,BR,71.82,light rain,-15.8900,-52.2567,Hotel Pousada Tropical
8,Lebu,CL,58.41,clear sky,-37.6167,-73.6500,Hostal Las Lilas
9,Lasa,IT,47.66,few clouds,46.6166,10.7002,Pension Feldgärtenhof
10,Castro,BR,59.86,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
12,Nkowakowa,ZA,60.85,overcast clouds,-23.8878,30.2871,Moytel 966A Guest House & Spa
13,Saint-Philippe,RE,76.55,broken clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""


In [24]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [25]:
# 8a. Create the output File (CSV)
output_data_file = "Weather_Database/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [37]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [38]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))